In [ ]:

# 📥 Instalar bibliotecas necessárias
!pip install python-docx lxml typing-extensions --quiet

# 📂 Preparar diretórios e baixar template
import os
from pathlib import Path

repo_dir = Path("/content/CDIA-ANVISA")
template_dir = repo_dir / "TCC - CDIA" / "Templates"
template_dir.mkdir(parents=True, exist_ok=True)

template_url = "https://raw.githubusercontent.com/LuckyLupa/CDIA-ANVISA/main/TCC%20-%20CDIA/Templates/modelo_relatorio_v2.docx"
template_path = template_dir / "modelo_relatorio_v2.docx"
os.system(f"wget -q {template_url} -O {template_path}")

print("✅ Template baixado com sucesso:", template_path)

# 🧾 Dados simulados da fatura (substitua com dados reais em produção)
dados_fatura = {
    "MES_ANO": "Julho/2025",
    "VALOR_TOTAL": "R$ 5.430,80",
    "PERIODO_MEDIDO": "01/07/2025 a 31/07/2025",
    "UNIDADE": "Galeão - RJ",
    "OBJETO": "Ressarcimento mensal de serviços de TIC",
    "RESPONSAVEL": "Luciano",
    "HASH_PDF": "a3f5c8d9b4e8f72c4569e1a3d8c9f02aab89c1f234abcd56789ef01234567890"
}

# 📝 Gerar o relatório a partir do template
from docx import Document

def gerar_relatorio(dados, caminho_template, caminho_saida):
    doc = Document(caminho_template)
    for p in doc.paragraphs:
        for chave, valor in dados.items():
            if f"{{{{{chave}}}}}" in p.text:
                p.text = p.text.replace(f"{{{{{chave}}}}}", valor)
    doc.save(caminho_saida)
    print("✅ Relatório gerado:", caminho_saida)

relatorio_nome = "relatorio_aux_riogaleao_tic_2025-07.docx"
output_path = repo_dir / "TCC - CDIA" / "TCC" / relatorio_nome
output_path.parent.mkdir(parents=True, exist_ok=True)

gerar_relatorio(dados_fatura, template_path, output_path)

# 🧠 Inicializar Git (se necessário) e adicionar remote
import getpass
token = getpass.getpass("Digite seu token GitHub: ")

os.chdir(repo_dir)
if not (repo_dir / ".git").exists():
    !git init
    !git remote add origin https://{token}@github.com/LuckyLupa/CDIA-ANVISA.git
    !git fetch origin
    !git checkout -b main
else:
    !git remote set-url origin https://{token}@github.com/LuckyLupa/CDIA-ANVISA.git

# 🚀 Commit e push com pull prévio para evitar conflitos
relatorio_path_repo = f"TCC - CDIA/TCC/{relatorio_nome}"

!git add "{relatorio_path_repo}"
!git commit -m "Atualiza relatório auxiliar TIC Julho/2025" || echo "Nada a commitar"
!git pull origin main --rebase
!git push origin main

# 🔗 Exibir link para o arquivo no GitHub
usuario = "LuckyLupa"
repo = "CDIA-ANVISA"
branch = "main"

relatorio_url = f"https://github.com/{usuario}/{repo}/blob/{branch}/{relatorio_path_repo.replace(' ', '%20')}"
print("📂 Relatório disponível em:", relatorio_url)
